In [1]:
import cv2
import time
import mediapipe as mp
import tkinter as tk
from threading import Thread


class FaceMeshDetector():
    def __init__(self, staticMode=False, maxFaces=3, minDetectionCon=0.5, minTrackCon=0.5, thickness=1, circle_radius=2):
        self.staticMode = staticMode
        self.maxFaces = maxFaces
        self.minDetectionCon = minDetectionCon
        self.minTrackCon = minTrackCon
        self.thickness = thickness
        self.circle_radius = circle_radius
        
        self.mpDraw = mp.solutions.drawing_utils
        self.mpFaceMesh = mp.solutions.face_mesh
        self.faceMesh = self.mpFaceMesh.FaceMesh(static_image_mode=self.staticMode, max_num_faces=self.maxFaces)
        self.drawSpec = self.mpDraw.DrawingSpec(color=(0, 255, 0), thickness=self.thickness, circle_radius=self.circle_radius)

    def findFaceMesh(self, img, draw=True):
        self.imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.faceMesh.process(self.imgRGB)
        faces = []
        if self.results.multi_face_landmarks:
            for faceLms in self.results.multi_face_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, faceLms, self.mpFaceMesh.FACEMESH_TESSELATION, self.drawSpec, self.drawSpec)
                face = []
                for id, lm in enumerate(faceLms.landmark):
                    ih, iw, ic = img.shape
                    x, y = int(lm.x * iw), int(lm.y * ih)
                    face.append([x, y])
                faces.append([face])
        return img, faces


def webcam_feed():
    global running
    running = True
    cap = cv2.VideoCapture(0)
    pTime = 0
    detector = FaceMeshDetector()
    while running:
        success, img = cap.read()
        if not success:
            print("Video playback completed or file not found.")
            break

        img, faces = detector.findFaceMesh(img)
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        cv2.putText(img, f'FPS: {int(fps)}', (10, 35), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 255), 2)
        cv2.putText(img, f'FaceMesh:', (10, 55), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 255), 2)
        cv2.imshow("Image", img)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


def start_webcam():
    global thread
    thread = Thread(target=webcam_feed)
    thread.start()


def stop_webcam():
    global running
    running = False
    if thread.is_alive():
        thread.join()


def exit_app():
    stop_webcam()
    window.destroy()


# GUI
window = tk.Tk()
window.title("Face Mesh Detector")

frame = tk.Frame(window)
frame.pack(pady=20)

btn_start = tk.Button(frame, text="Start", command=start_webcam)
btn_start.pack(side=tk.LEFT, padx=10)

btn_stop = tk.Button(frame, text="Stop", command=stop_webcam)
btn_stop.pack(side=tk.LEFT, padx=10)

btn_exit = tk.Button(frame, text="Exit", command=exit_app)
btn_exit.pack(side=tk.LEFT, padx=10)

window.mainloop()